# YOLOv8 Fine-Tuning for Safety Detection (Hard Hats & Vests)

This notebook demonstrates how to fine-tune a YOLOv8 model to detect safety equipment using the Hard Hat Workers dataset from Roboflow.

## Step 1: Install Dependencies

In [ ]:
!pip install ultralytics roboflow

## Step 2: Download the Dataset

### 🔑 How to find your Roboflow API Key:
1. Log in to [Roboflow](https://app.roboflow.com/)
2. Click your **Profile Picture** (top right) > **Settings**
3. On the left sidebar, select **Workspaces**
4. Select your workspace and click **Roboflow API**
5. Copy your **Private API Key** and paste it below.

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_PRIVATE_API_KEY")
project = rf.workspace("pedro-olivares-1lezt").project("hard-hat-workers-dataset")
version = project.version(1)
dataset = version.download("yolov8")

## Step 3: Train the Model

We use the `yolov8n.pt` (nano) model as a base for transfer learning.

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('yolov8n.pt')

# Train the model
results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=50,
    imgsz=640,
    plots=True
)

## Step 4: Validate and Evaluate

Check the mAP50 and other metrics.

In [ ]:
metrics = model.val()
print(f"mAP50: {metrics.box.map50}")

## Step 5: Run Inference on a Sample Image

In [ ]:
import cv2
import os
from google.colab.patches import cv2_imshow

# Run prediction on an image from the test set
test_images_path = os.path.join(dataset.location, 'test', 'images')
test_images = os.listdir(test_images_path)
img_path = os.path.join(test_images_path, test_images[0])

results = model.predict(source=img_path, save=True)
res_img = results[0].plot()
cv2_imshow(res_img)

## Step 6: Export the Model

Export to ONNX format or just keep the `best.pt` for deployment.

In [ ]:
model.export(format='onnx') # Optional: Export for other platforms